In [1]:
!pip install data-purifier

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/12.8 MB ? eta -:--:--
                                             0.1/12.8 MB 871.5 kB/s eta 0:00:15
                                              0.2/12.8 MB 1.6 MB/s eta 0:00:08
     -                                        0.4/12.8 MB 2.8 MB/s eta 0:00:05
     --                                       0.9/12.8 MB 4.9 MB/s eta 0:00:03
     -----                                    1.8/12.8 MB 7.3 MB/s eta 0:00:02
     -------                                  2.3/12.8 MB 8.5 MB/s eta 0:00:02
     -------                                  2.3/12.8 MB 8.5 MB/s eta 0:00:02
     --------                                 2.6/12.8 MB 6.7 MB/s eta 0:00:02
     ---------                                2.9/12.8 MB 7.1 MB/s eta 0:00:02
     ----------                               3.3/12.8 MB 6.8 MB/s eta 0:00:02
     ----------                               3.3/12.8 MB 6

In [3]:
# importing packages

import datapurifier as dp
from datapurifier import Mleda
from datapurifier import Nlpeda
from datapurifier import Nlpurifier

import seaborn as sns
import pandas as pd
import numpy as np

print(dp.__version__)

0.3.6


In [14]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

#from google.colab import drive

# drive.mount('/content/drive')

# path_link = "/content/drive/MyDrive/train.csv"
df = pd.read_csv('train.csv')

In [15]:
df.shape

(630000, 11)

In [16]:
columns_to_drop = ['user_id', 'book_id','date_added',
       'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments']
df.drop(columns=columns_to_drop, inplace=True)

In [17]:
pure = Nlpurifier(df, "review_text")

GridspecLayout(children=(Checkbox(value=False, description='Drop Null Rows', indent=False, layout=Layout(grid_…


Convert Word to its Base Form


interactive(children=(RadioButtons(description='Technique:', options=('None', 'Stemming', 'Lemmatization'), va…

Remove Top Common Words


interactive(children=(Checkbox(value=False, description='Remove Top Common Words'), Output()), _dom_classes=('…

Remove Top Rare Words


interactive(children=(Checkbox(value=False, description='Remove Top Rare Words'), Output()), _dom_classes=('wi…

Button(description='Start Purifying', style=ButtonStyle())

Dataframe contains 630000 rows and 3 columns

Starting execution of 'lower' function, Please wait...
Finished execution of 'lower' function in 1.667 secs

Starting execution of 'remove_numbers' function, Please wait...


In [9]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')

nltk.download('wordnet')
pure_df=pure.df

# Initializing the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization on a sentence
def lemmatize_sentence(sentence):
    tokens = word_tokenize(sentence)  # Tokenize the sentence into words
    lemmatized_sentence = ' '.join([lemmatizer.lemmatize(word) for word in tokens])  # Lemmatize each word
    return lemmatized_sentence

# Applying lemmatization to the 'review' column
pure_df['review_text'] = pure_df['review_text'].apply(lambda x: lemmatize_sentence(x))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
print(pure_df)

                               review_id  \
0       55d9037adc00645b17f0cf43fa4d053e   
1       e5f389a112b12d560e42e0c5c3d4eb35   
2       e7892549ac69653aceaac86760575aa0   
3       ab4acc9ad0a24a072356c331880b83d5   
4       0b65c86a90814e4ab5db50e0cd3dc808   
...                                  ...   
629995  f71a90adff16ad1e0bddd8783775bb42   
629996  cb562651157caa44e46118930d74edf3   
629997  4b49110a886bd8f14a256bb702908578   
629998  276575dd94bc9547526829a303f56ae6   
629999  55ab6de1a263d6f33cc9e83c71f727e9   

                                              review_text  rating  
0       read audiobook book follows primarily jack pri...       3  
1       easily best sci-fi book kid didnt think suzann...       5  
2       best sarah dessen book ever say book suck knuc...       4  
3       series get better better scarlet didnt disappo...       5  
4       review hottest underhyped book ya lit enter ch...       5  
...                                                   ...     .

In [11]:
pure_df.to_csv('train1.csv', index=False)

In [12]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
df=pd.read_csv("/content/train1.csv")

In [13]:
df.drop('review_id',axis=1,inplace=True)

In [14]:
df['review_text'].fillna(method='ffill', inplace=True)

In [15]:
missing_values = df.isnull().sum()          # show no. ōf nulls in each column
print(missing_values)

review_text    0
rating         0
dtype: int64


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df['review_text'].tolist()
tfidf_vectorizer=TfidfVectorizer(max_features=55000)
tfidf_features=tfidf_vectorizer.fit_transform(corpus)

In [17]:
X=tfidf_features
y=pure_df['rating']

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# from sklearn.linear_model import LogisticRegression
# from sklearn import metrics
# #Create a Logistic Regression model
# model = LogisticRegression()

# #Train the model on the training set
# model.fit(X_train, y_train)

# #Make predictions on the test set
# y_pred = model.predict(X_test)

# #Evaluate the model
# accuracy = metrics.accuracy_score(y_test, y_pred)
# print(f'Accuracy:{accuracy}')

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

class_weights = {0:1, 1:1.5, 2:2 ,3:4 ,4:5.5 ,5:4.5}  #assigning weights to data

# Create a Logistic Regression model with class weights
model = LogisticRegression(class_weight=class_weights)

model.fit(X_train, y_train)

#Make predictions on the test set
y_pred = model.predict(X_test)

#Evaluate the model
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy:{accuracy}')

Accuracy:0.5083015873015873


In [22]:
from sklearn.metrics import f1_score

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {f1}')


F1 Score: 0.48684088037435475


In [23]:
# from sklearn.svm import SVC
# from sklearn import metrics
# # Create an SVM classifier
# svm_classifier = SVC()

# # Train the classifier on the training data
# svm_classifier.fit(X_train, y_train)

# # Make predictions on the testing data
# y_pred = svm_classifier.predict(X_test)

# # Evaluate the accuracy of the classifier
# accuracy = metrics.accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy * 100:.2f}%')

In [24]:
path_link = "/content/drive/MyDrive/test.csv"
df1 = pd.read_csv(path_link)

In [25]:
columns_to_drop = ['user_id', 'book_id','date_added',
       'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments']
df1.drop(columns=columns_to_drop, inplace=True)

In [26]:
pure1 = Nlpurifier(df1, "review_text")

GridspecLayout(children=(Checkbox(value=False, description='Drop Null Rows', indent=False, layout=Layout(grid_…


Convert Word to its Base Form


interactive(children=(RadioButtons(description='Technique:', options=('None', 'Stemming', 'Lemmatization'), va…

Remove Top Common Words


interactive(children=(Checkbox(value=False, description='Remove Top Common Words'), Output()), _dom_classes=('…

Remove Top Rare Words


interactive(children=(Checkbox(value=False, description='Remove Top Rare Words'), Output()), _dom_classes=('wi…

Button(description='Start Purifying', style=ButtonStyle())

Dataframe contains 270000 rows and 2 columns

Starting execution of 'lower' function, Please wait...
Finished execution of 'lower' function in 0.577 secs

Starting execution of 'remove_numbers' function, Please wait...
Finished execution of 'remove_numbers' function in 3.455 secs

Starting execution of 'remove_html_tags' function, Please wait...
Finished execution of 'remove_html_tags' function in 18.231 secs

Starting execution of 'remove_emails' function, Please wait...
Finished execution of 'remove_emails' function in 14.002 secs

Starting execution of 'remove_urls' function, Please wait...
Finished execution of 'remove_urls' function in 3.113 secs

Starting execution of 'remove_multiple_spaces' function, Please wait...
Finished execution of 'remove_multiple_spaces' function in 4.839 secs

Starting execution of 'remove_accented_chars' function, Please wait...
Finished execution of 'remove_accented_chars' function in 2.491 secs

Starting execution of 'remove_stop_words' function, Ple

In [28]:
pure_df1=pure1.df

# Initializing the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization on a sentence
def lemmatize_sentence(sentence):
    tokens = word_tokenize(sentence)  # Tokenize the sentence into words
    lemmatized_sentence = ' '.join([lemmatizer.lemmatize(word) for word in tokens])  # Lemmatize each word
    return lemmatized_sentence

# Applying lemmatization to the 'review' column
pure_df1['review_text'] = pure_df1['review_text'].apply(lambda x: lemmatize_sentence(x))



In [29]:
pure_df1.to_csv('test1.csv', index=False)

In [30]:
df2=pd.read_csv("/content/test1.csv")

In [31]:
df2.drop('review_id',axis=1,inplace=True)

In [32]:
df2['review_text'].fillna(method='ffill', inplace=True)

In [33]:
missing_values = df2.isnull().sum()          # show no. ōf nulls in each column
print(missing_values)

review_text    0
dtype: int64


In [34]:
corpus1 = df2['review_text'].tolist()
tfidf_vectorizer1=TfidfVectorizer(max_features=55000)
tfidf_features1=tfidf_vectorizer.transform(corpus1)

In [35]:
X1=tfidf_features1

In [36]:
# Make predictions on the test set
y_pred = model.predict(X1)


In [37]:
path_link = "/content/drive/MyDrive/test.csv"
df4 = pd.read_csv(path_link)

predictions_df = pd.DataFrame(y_pred, columns=['rating'])

result_df = pd.concat([ df4['review_id'],predictions_df], axis=1)
# Save the modified DataFrame back to a CSV file
result_df.to_csv('submission_1.csv',index=False)